# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [173]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import traceback
import gmaps
import os

# Import API key
from api_keys import g_key

In [ ]:
### Store Part I results into DataFrame
# * Load the csv exported in Part I to a DataFrame

In [174]:
city_data_df = pd.read_csv("City_Data.csv", index_col=False)
city_data_df.reset_index(drop=True, inplace=True)
city_data_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,georgetown,5.41,100.34,88.25,74,20,10.29,MY,2020-05-15 04:48:15
1,punta arenas,-53.15,-70.92,37.20,93,20,4.70,CL,2020-05-15 04:49:58
2,cabo san lucas,22.89,-109.91,75.40,44,0,16.11,MX,2020-05-15 04:45:14
3,mahebourg,-20.41,57.70,71.60,83,20,5.82,MU,2020-05-15 04:49:58
4,am timan,11.03,20.28,83.53,39,95,7.09,TD,2020-05-15 04:49:58
...,...,...,...,...,...,...,...,...,...
557,penzance,50.12,-5.54,44.94,87,0,10.29,GB,2020-05-15 04:52:33
558,tecoanapa,16.52,-98.75,79.59,70,0,2.26,MX,2020-05-15 04:52:34
559,kupang,-10.17,123.58,87.80,62,75,21.92,ID,2020-05-15 04:51:43
560,kazachinskoye,57.70,93.28,55.31,42,100,5.75,RU,2020-05-15 04:52:34


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [175]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lng' into  locations 
locations = city_data_df[['Lat', 'Lng']].astype(float)

# Convert Humidity to float and store
humidity = city_data_df['Humidity'].astype(float)
print(len(locations))
print(len(humidity))

562
562


In [176]:
# Create a poverty Heatmap layer
starting_coordinates = (22.32, 114.17)
fig = gmaps.figure(center=starting_coordinates, 
                   zoom_level=4,
                   map_type='HYBRID')

heat_layer = gmaps.heatmap_layer(locations, 
                                 weights=humidity, 
                                 dissipating=False,         
                                 max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [177]:
temp_df = city_data_df[(city_data_df['Max Temp'] < 90)  & (city_data_df['Max Temp'] > 80)]
temp_df = temp_df[(temp_df['Humidity'] < 70)  & (temp_df['Humidity'] > 60)]
temp_df = temp_df[(temp_df['Wind Speed'] < 20)  & (temp_df['Wind Speed'] > 10)]
temp_df = temp_df.dropna()
temp_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
94,victoria,22.29,114.16,86.07,62,71,11.63,HK,2020-05-15 04:48:45
136,dzaoudzi,-12.79,45.27,82.40,65,40,19.46,YT,2020-05-15 04:50:34
148,wanning,18.80,110.40,88.75,63,3,12.88,CN,2020-05-15 04:50:37
194,hobyo,5.35,48.53,84.81,67,2,19.53,SO,2020-05-15 04:50:49
461,keti bandar,24.14,67.45,87.49,62,16,10.80,PK,2020-05-15 04:52:04
487,karaton,-6.34,106.10,89.28,66,20,14.99,ID,2020-05-15 04:52:12


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [178]:
lat = []
lng = []
hotels = []
cities = []
countries = []
vicinities = []
ratings = []

for index, row in temp_df.iterrows():
    print("--------")
    print(row['City'])

    # geocoordinates
    target_coordinates = (f"{row['Lat']}, {row['Lng']}")
    target_search = "hotel"
    target_radius = 5000
    target_type = "hotel"
    target_rank = "rating"

    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "rankBy": target_rank,
        "key": g_key
    }

    try:
        # base url
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        # run a request using our params dictionary
        response = requests.get(base_url, params=params)
        #print(response.url)
        # convert response to json
        results = response.json()
        #print(response.json())
        # Print the json (pretty printed)
        #print(json.dumps(results, indent=4, sort_keys=True))

        hotels.append(results["results"][0]["name"])
        lat.append(row['Lat'])
        lng.append(row['Lng'])
        cities.append(row['City'])
        countries.append(row['Country'])
        vicinities.append(results["results"][0]["vicinity"])
        ratings.append(results["results"][0]["rating"])

        print(results["results"][0]["rating"])
        print(results["results"][0]["name"])
        print(results["results"][0]["vicinity"])
        
    except Exception as e:
        print(f"###{e}###")
        print(f"### No hotel(S) found for city: {row['City']}. Skipping...")
        print(traceback.format_exc())

--------
victoria
4.1
Dorsett Mongkok, Hong Kong
88 Tai Kok Tsui Rd, Tai Kok Tsui
--------
dzaoudzi
3.3
Hotel Maharajah Mayotte
Mamoudzou
--------
wanning
0
Paradise Hotel
Wanning, Renmin Middle Rd, 万城镇人民中街
--------
hobyo
###list index out of range###
### No hotel(S) found for city: hobyo. Skipping...
Traceback (most recent call last):
  File "<ipython-input-178-4bff4fa02196>", line 43, in <module>
    hotels.append(results["results"][0]["name"])
IndexError: list index out of range

--------
keti bandar
###list index out of range###
### No hotel(S) found for city: keti bandar. Skipping...
Traceback (most recent call last):
  File "<ipython-input-178-4bff4fa02196>", line 43, in <module>
    hotels.append(results["results"][0]["name"])
IndexError: list index out of range

--------
karaton
3.8
OYO 3173 Wisma Pkp-ri
Jalan Raya Labuan Km.3 Majasari, Sukaratu, Kabupaten Pandeglang


In [179]:
hotel_df = pd.DataFrame({'Hotel Name':hotels,
                        'City':cities,
                        'Country':countries,
                        'Lat':lat,
                        'Lng':lng,
                        'Vicinity':vicinities,
                        'Rating':ratings})
hotel_df
#hotel_df.sort_values(['Country'], ascending=True)

,Hotel Name,City,Country,Lat,Lng,Vicinity,Rating
0,"Dorsett Mongkok, Hong Kong",victoria,HK,22.29,114.16,"88 Tai Kok Tsui Rd, Tai Kok Tsui",4.1
1,Hotel Maharajah Mayotte,dzaoudzi,YT,-12.79,45.27,Mamoudzou,3.3
2,Paradise Hotel,wanning,CN,18.80,110.40,"Wanning, Renmin Middle Rd, 万城镇人民中街",0.0
3,OYO 3173 Wisma Pkp-ri,karaton,ID,-6.34,106.10,"Jalan Raya Labuan Km.3 Majasari, Sukaratu, Kab...",3.8


In [181]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [182]:
# Add marker layer ontop of heat map
# Create hotel symbol layer
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=4,
    info_box_content=hotel_info
)


#fig = gmaps.figure()
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

fig

# Display figure


Figure(layout=FigureLayout(height='420px'))